[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](YOUR_GITHUB_COLAB_URL_HERE)


# Chapter 1, Input: Load C3D, TRC, and CSV in Google Colab

Ebook: A Hands-On Guide to Biomechanics Data Analysis with Python and AI  
Author: Dr. Hossein Mokhtarzadeh  
Powered by PoseIQ™

This notebook loads sample biomechanics data and shows how to bring it into Python.  
Click Runtime → Run all.


## Install packages

In [ ]:
!pip -q install ezc3d pandas

## Download sample data from c3d.org

In [ ]:
import os, urllib.request, zipfile

url = "https://www.c3d.org/data/Sample01.zip"
zip_path = "sample_data.zip"
urllib.request.urlretrieve(url, zip_path)

os.makedirs("sample_data", exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall("sample_data")

print("Extracted files in sample_data")
for root, dirs, files in os.walk("sample_data"):
    for f in files:
        print(os.path.join(root, f))


## Load a C3D file with ezc3d

In [ ]:
import os, ezc3d, numpy as np

c3d_files = []
for root, dirs, files in os.walk("sample_data"):
    for f in files:
        if f.lower().endswith(".c3d"):
            c3d_files.append(os.path.join(root, f))

if not c3d_files:
    raise FileNotFoundError("No C3D files found.")

c3d_path = c3d_files[0]
print("Loading C3D:", c3d_path)
c3d = ezc3d.c3d(c3d_path)

points = c3d["data"]["points"]
analogs = c3d["data"]["analogs"]
point_labels = c3d["parameters"]["POINT"]["LABELS"]["value"]
analog_labels = c3d["parameters"]["ANALOG"]["LABELS"]["value"] if "ANALOG" in c3d["parameters"] else []

print("Points shape:", points.shape)
print("Analogs shape:", analogs.shape if hasattr(analogs, "shape") else "not available")
print("First 10 point labels:", point_labels[:10])
print("First 10 analog labels:", analog_labels[:10] if analog_labels else [])


## Plot the first analog channel if available

In [ ]:
import matplotlib.pyplot as plt

if isinstance(analogs, np.ndarray) and analogs.size > 0 and analog_labels:
    analog_rate = float(c3d["parameters"]["ANALOG"]["RATE"]["value"])
    num_channels = analogs.shape[0]
    num_frames = analogs.shape[2]
    t = np.arange(num_frames) / analog_rate

    plt.figure(figsize=(10, 4 * num_channels))  # Adjust figure size based on number of channels

    for ch in range(num_channels):
        y = analogs[ch, 0, :]  # Select data for a single channel and one component (assuming 1 component per channel)
        plt.subplot(num_channels, 1, ch + 1) # Create subplots for each channel
        plt.plot(t, y)
        plt.xlabel("Time (s)")
        plt.ylabel(analog_labels[ch])

        plt.title(f"Analog quick plot - Channel {ch+1}: {analog_labels[ch]}")
        plt.grid(True)

    plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
    plt.show()
else:
    print("No analog data available to plot.")

## Load a TRC or CSV into pandas

In [ ]:
import pandas as pd

trc_path, csv_path = None, None
for root, dirs, files in os.walk("sample_data"):
    for f in files:
        if f.lower().endswith(".trc") and trc_path is None:
            trc_path = os.path.join(root, f)
        if f.lower().endswith(".csv") and csv_path is None:
            csv_path = os.path.join(root, f)

df = None
if trc_path:
    print("Loading TRC:", trc_path)
    df = pd.read_csv(trc_path, sep="\t", skiprows=3, engine="python")
elif csv_path:
    print("Loading CSV:", csv_path)
    df = pd.read_csv(csv_path)
else:
    print("No TRC or CSV found, creating an example DataFrame")
    df = pd.DataFrame({
        "Time": [0.0, 0.01, 0.02, 0.03],
        "CoM_Z": [0.88, 0.881, 0.879, 0.880]
    })

print(df.head())


## Summary

You can now:
1. Load a C3D file with ezc3d  
2. Preview analog channels and plot a quick signal  
3. Load TRC or CSV into pandas and inspect the data

Next, we will parse and clean the signals so they are analysis-ready.
